In [1]:
#!/usr/bin/Rscript
#############################################################
options(stringsAsFactors = FALSE)
##############################################################

suppressMessages(
    {

    library(tidyverse)
    library(data.table)
    library(dynamicTreeCut)
    library(fastcluster)
    library(WGCNA)
    library(flashClust)
    library(lattice)
    library(latticeExtra)
    library(gridExtra)
    library(grid)
    library(ggplot2)
    library(IRdisplay)

    #enableWGCNAThreads(15)
    #change that on rf    
    allowWGCNAThreads(15)

    }
                )


save_plot <- function(p, fn, w, h){
    for(ext in c(".pdf", ".png")){
        ggsave(filename=paste0(fn,ext), plot=p, width=w, height=h)
    }
}


dir.create('../_m/')
setwd('../_m/')

Allowing multi-threading with up to 15 threads.


Warning message in dir.create("../_m/"):
“'../_m' already exists”


In [2]:

counts_path <- Sys.glob('../../../../jhpce_data/analysis/*/residualized_expression.tsv')
counts_path


region <- gsub('.*analysis/|/residu.*','',counts_path)
region

[1] "../../../../jhpce_data/analysis/caudate/residualized_expression.tsv"    
[2] "../../../../jhpce_data/analysis/dlpfc/residualized_expression.tsv"      
[3] "../../../../jhpce_data/analysis/hippocampus/residualized_expression.tsv"

[1] "caudate"     "dlpfc"       "hippocampus"

In [3]:
rdata_path <- Sys.glob('../../../../jhpce_data/analysis/*/voomSVA.RData')
rdata_path
#load(rdata_path)

[1] "../../../../jhpce_data/analysis/caudate/voomSVA.RData"    
[2] "../../../../jhpce_data/analysis/dlpfc/voomSVA.RData"      
[3] "../../../../jhpce_data/analysis/hippocampus/voomSVA.RData"

In [4]:
#process the dataframes
process_data <- function(data, group) {
  data %>%
        dplyr::select(row.names(group)) %>%
        t() %>%
        .[goodSamplesGenes(., verbose = 3)$goodSamples, goodSamplesGenes(., verbose = 3)$goodGenes] %>%
        t()
    }

# split the expr data between groups (ie. case vs control, male vs female etc)
separate_data <- function(data,group) {
    df_tmp <- table_all %>% 
                        t() %>%
                        as.data.frame() %>%
                        tibble::rownames_to_column(var = 'row_name') %>%
                        filter(row_name %in% row.names(group)) %>%
                        tibble::column_to_rownames(var = 'row_name')
    return(df_tmp)
    
    }

# Define a function to get the max value between two vectors
max_vector <- function(x, y) { 
    if (max(x) > max(y)) x else y
        }

In [5]:
#c('group_A','group_B')
setLabels <- c('Male','Female')
setA <- setLabels[1]
setB <- setLabels[2]

In [6]:
scale_free_df = data.frame()

In [7]:

for (i in seq_along(counts_path)){
    
    load(rdata_path[i])
    
    dir.create(region[i])
    
    metadata <- v$targets %>% 
                        as.data.frame()
    
    CTL <- metadata %>% 
                    filter(Dx == 'Control')
    SZD <- metadata %>% 
                    filter(Dx != 'Control')
    CTL_SZD <- metadata

    
    for (sample_filter in c('CTL', 'SZD', 'CTL_SZD')){
        
        save_path <- paste0(region[i],'/',sample_filter,'/')
        print(save_path)
        
        dir.create(save_path)
        
        filtered_metadata <- get(sample_filter)
        
        group_a <- filtered_metadata %>% 
                                    filter(Sex == 'M') #male only
        group_b <- filtered_metadata %>% 
                                    filter(Sex != 'M') #female only   
    
        vsd<-fread(counts_path[i],header = T, na.strings = "",check.names = F) %>%
                                                tibble::column_to_rownames(var = "feature_id") #%>% slice_head(n=1000)
    
        
        # Process data for group_a and group_b
        vsd_group_a <- process_data(vsd, group_a)
        vsd_group_b <- process_data(vsd, group_b)

        # Merge the processed data
        table_all <- merge(vsd_group_a, vsd_group_b, by = "row.names") %>%
                                                tibble::column_to_rownames('Row.names') 


        datExprA <- separate_data(tabble_all, group_a)
        datExprB <- separate_data(tabble_all, group_b)
        datExprAll <- table_all %>%
                                t() %>%
                                as.data.frame()

        sampleTreeA <- flashClust(dist(datExprA), method="average")
        sampleTreeB <- flashClust(dist(datExprB), method="average")
        sampleTreeAll <- flashClust(dist(datExprAll), method="average")


        ###save as .PDF file 
        pdf(paste0(save_path,'1a-Dist_clust_',sample_filter,'.pdf'),height=10,width = 15)
        par(mar=c(2,4,1,0), mfrow=c(1,2), oma=c(2,0,4,0), cex=0.5)
        plot(sampleTreeA, main=paste0(setA, ' ', sample_filter), sub="", cex.lab=1.2, cex.axis=1.2, cex.main=1.7, ylim=20)
        plot(sampleTreeB, main=paste0(setB, ' ',sample_filter), sub="", cex.lab=1.2, cex.axis=1.2, cex.main=1.7, ylim=20)
        par(cex=1)
        title(main="Sample Clustering Based on Distance ",outer=TRUE)
        dev.off()


        ###save as .PDF file 
        pdf(paste0(save_path,'1a-Dist_clust_all_',sample_filter,'.pdf'),height=10,width = 15)
        par(cex=0.5)
        plot(sampleTreeAll, main='', sub="", cex.lab=1.2, cex.axis=1.2, cex.main=1.7, ylim=20)
        title(main="Sample Clustering Based on Distance ",outer=TRUE)
        dev.off()
        
        
        powers1 = c(seq(1,30,by = 1))
        sink('pickSoftThreshold.log')
        RpowerTableA <- pickSoftThreshold(datExprA, powerVector = powers1, RsquaredCut = 0.85, verbose = 1)
        RpowerTableB <- pickSoftThreshold(datExprB, powerVector = powers1, RsquaredCut = 0.85, verbose = 1)
        sink()
        
        
        male_sfa <- RpowerTableA$fitIndices
        male_sfa$brain_region <- region[i]
        male_sfa$sex <- 'male'
        male_sfa$status <- sample_filter
        
        female_sfa <- RpowerTableB$fitIndices
        female_sfa$brain_region <- region[i]
        female_sfa$sex <- 'female'
        female_sfa$status <- sample_filter
        
        male_female_scalefree <- rbind(male_sfa, female_sfa) %>% filter(Power == 15)
        
                
        display('male')
        display(RpowerTableA$fitIndices)
        display('female')
        display(RpowerTableB$fitIndices)

        
        #save results into a dataframe
        RpowerTableA$fitIndices %>%
                fwrite(paste0(save_path,setA,'_scale_free_metrics.tsv'),quote=F,sep='\t',row.names=F)
        RpowerTableB$fitIndices %>%
                fwrite(paste0(save_path,setB,'_scale_free_metrics.tsv'),quote=F,sep='\t',row.names=F)
        
        
        
        
        scale_free_df <- rbind(male_female_scalefree,scale_free_df)
                
        
        print(dim(datExprAll))
        
        
    
        }
    
    }

Loading required package: limma



[1] "caudate/CTL/"
 Flagging genes and samples with too many missing values...
  ..step 1
 Flagging genes and samples with too many missing values...
  ..step 1
 Flagging genes and samples with too many missing values...
  ..step 1
 Flagging genes and samples with too many missing values...
  ..step 1


[1] "male"

Power,SFT.R.sq,slope,truncated.R.sq,mean.k.,median.k.,max.k.
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.3496741,-2.024912,0.8822997,2.806856e+03,2.741132e+03,5080.035186
2,0.6608887,-2.174302,0.9346218,5.126239e+02,4.469268e+02,1619.786125
3,0.7208429,-2.208397,0.9387765,1.315792e+02,9.390649e+01,689.371365
4,0.7825026,-2.122364,0.9655373,4.311499e+01,2.336145e+01,353.849053
5,0.8300185,-2.058906,0.9732297,1.697152e+01,6.562800e+00,206.418402
6,0.8884784,-1.958186,0.9884414,7.691604e+00,2.024853e+00,131.753267
7,0.9226762,-1.875098,0.9926598,3.892400e+00,6.845366e-01,90.291854
8,0.9056655,-1.884634,0.9756146,2.150197e+00,2.471844e-01,67.473358
9,0.9196377,-1.813267,0.9797545,1.274424e+00,9.403218e-02,51.980018


[1] "female"

...

[1] "male"

Power,SFT.R.sq,slope,truncated.R.sq,mean.k.,median.k.,max.k.
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.02909356,-0.3335507,0.9346659,4.036498e+03,4.003268e+03,7729.978102
2,0.54360659,-1.3334790,0.9279818,1.047745e+03,9.410015e+02,3292.438248
3,0.70388720,-1.6208922,0.9431905,3.660994e+02,2.783912e+02,1691.063128
4,0.77625896,-1.7146893,0.9636949,1.552171e+02,9.584107e+01,974.617306
5,0.80392103,-1.7785377,0.9713541,7.536090e+01,3.674948e+01,617.188594
6,0.81066857,-1.8261414,0.9702699,4.042582e+01,1.514688e+01,416.015698
7,0.82064536,-1.8334589,0.9752038,2.340069e+01,6.702155e+00,293.775934
8,0.82823691,-1.8230833,0.9780410,1.438074e+01,3.098788e+00,215.119327
9,0.83171530,-1.8124508,0.9771551,9.273508e+00,1.499254e+00,162.162020


[1] "female"

Power,SFT.R.sq,slope,truncated.R.sq,mean.k.,median.k.,max.k.
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.09962681,-0.5236098,0.9315180,4709.7351951,4.459442e+03,9216.34274
2,0.71818034,-1.2948392,0.9682178,1382.8647319,1.133411e+03,4520.70377
3,0.83636771,-1.5090523,0.9657981,533.2447488,3.566499e+02,2610.49360
4,0.87238527,-1.5856279,0.9695112,244.8242769,1.288883e+02,1663.44443
5,0.88478687,-1.6095379,0.9722012,126.8791974,5.112581e+01,1133.44875
6,0.88615290,-1.6187610,0.9729478,71.8480169,2.192256e+01,811.52168
7,0.88885858,-1.6186377,0.9755335,43.5243559,1.011302e+01,603.51580
8,0.89100244,-1.6171027,0.9780732,27.8005235,4.868466e+00,461.88913
9,0.89305980,-1.6135541,0.9810670,18.5309851,2.425969e+00,361.71239


[1]   375 26727


In [8]:
scale_free_df

Power,SFT.R.sq,slope,truncated.R.sq,mean.k.,median.k.,max.k.,brain_region,sex,status
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
15,0.8603353,-1.6496386,0.9721536,1.2683011,0.0356344068,43.80547,hippocampus,male,CTL_SZD
15,0.9129470,-1.5555803,0.9899227,2.8848024,0.0712492655,114.24422,hippocampus,female,CTL_SZD
15,0.8998215,-1.7202108,0.9895271,1.6997666,0.0819433563,72.02038,hippocampus,male,SZD
15,0.8667655,-1.8004127,0.9775726,3.3379508,0.2572405980,133.03247,hippocampus,female,SZD
15,0.8503683,-1.7506404,0.9813398,1.5508506,0.0557427110,51.48229,hippocampus,male,CTL
15,0.8814454,-1.5697381,0.9728334,4.7189026,0.1549152156,151.23476,hippocampus,female,CTL
15,0.8758955,-0.6907531,0.8932539,171.9578570,0.0732892874,1833.11636,dlpfc,male,CTL_SZD
15,0.7950718,-1.4876494,0.9513087,8.7369342,0.1303790612,196.73970,dlpfc,female,CTL_SZD
15,0.7395474,-0.5961472,0.9677738,383.2604547,0.2555396597,2963.91928,dlpfc,male,SZD


In [9]:
dev.off()

null device 
          1

### We didn't achievied a scale-free topology index (R^2 >= 0.85) on some DLPFC networks using all samples from this brain region, so let's try to remove some possible outliers samples based on  DLPFC hierarchical dendograms ('_m/dlpfc/{CTL,SZD}/*.pdf'), lets try to remove some outliers samples to try to achieve a scale free network

In [10]:
# run script again, but only with dlpfc data, removing 3 DLPFC samples

scale_free_df2 = data.frame()


for (i in 2){ #dlpfc expression data is the second vector from counts_path
    
    load(rdata_path[i])
    
    dir.create(region[i])
    
    metadata <- v$targets %>% 
                        as.data.frame()
    
    CTL <- metadata %>% 
                    filter(Dx == 'Control')
    SZD <- metadata %>% 
                    filter(Dx != 'Control')
    CTL_SZD <- metadata

    
    for (sample_filter in c('CTL', 'SZD', 'CTL_SZD')){
            
        save_path <- paste0(region[i],'/',sample_filter,'/')
        print(save_path)
        
        dir.create(save_path)
        
        filtered_metadata <- get(sample_filter)
        
        group_a <- filtered_metadata %>% 
                                    filter(Sex == 'M') %>% #male only
                                    filter(RNum != 'R3555') #remove this specific sample, huge outlier
        group_b <- filtered_metadata %>% 
                                    filter(Sex != 'M') %>% #female only 
                                    filter(!RNum %in% c('R12351', 'R12371'))  #remove these specific samples, outliers
    
        vsd<-fread(counts_path[i],header = T, na.strings = "",check.names = F) %>%
                                                tibble::column_to_rownames(var = "feature_id") #%>% slice_head(n=1000)
    
        
        # Process data for group_a and group_b
        vsd_group_a <- process_data(vsd, group_a)
        vsd_group_b <- process_data(vsd, group_b)

        # Merge the processed data
        table_all <- merge(vsd_group_a, vsd_group_b, by = "row.names") %>%
                                                tibble::column_to_rownames('Row.names') 


        datExprA <- separate_data(tabble_all, group_a)
        datExprB <- separate_data(tabble_all, group_b)
        datExprAll <- table_all %>%
                                t() %>%
                                as.data.frame()

        sampleTreeA <- flashClust(dist(datExprA), method="average")
        sampleTreeB <- flashClust(dist(datExprB), method="average")
        sampleTreeAll <- flashClust(dist(datExprAll), method="average")

        
        powers1 = c(seq(1,30,by = 1))
        # sink('pickSoftThreshold.log')
        RpowerTableA <- pickSoftThreshold(datExprA, powerVector = powers1, RsquaredCut = 0.85, verbose = 1)
        RpowerTableB <- pickSoftThreshold(datExprB, powerVector = powers1, RsquaredCut = 0.85, verbose = 1)
        # sink()
        
        
        male_sfa <- RpowerTableA$fitIndices
        male_sfa$brain_region <- region[i]
        male_sfa$sex <- 'male'
        male_sfa$status <- sample_filter
        
        female_sfa <- RpowerTableB$fitIndices
        female_sfa$brain_region <- region[i]
        female_sfa$sex <- 'female'
        female_sfa$status <- sample_filter
        
        male_female_scalefree <- rbind(male_sfa, female_sfa) %>% filter(Power == 15)
        
                
        display('male')
        display(RpowerTableA$fitIndices)
        display('female')
        display(RpowerTableB$fitIndices)

        
        RpowerTableA$fitIndices #%>%
                #fwrite(paste0(save_path,setA,'_scale_free_metrics.tsv'),quote=F,sep='\t',row.names=F)
        RpowerTableB$fitIndices #%>%
                #fwrite(paste0(save_path,setB,'_scale_free_metrics.tsv'),quote=F,sep='\t',row.names=F)
        
        
        
        
        scale_free_df2 <- rbind(male_female_scalefree,scale_free_df2)
        
        
        print(dim(datExprAll))
                
    
        }
    
    }

Warning message in dir.create(region[i]):
“'dlpfc' already exists”


[1] "dlpfc/CTL/"


Warning message in dir.create(save_path):
“'dlpfc/CTL' already exists”


 Flagging genes and samples with too many missing values...
  ..step 1
 Flagging genes and samples with too many missing values...
  ..step 1
 Flagging genes and samples with too many missing values...
  ..step 1
 Flagging genes and samples with too many missing values...
  ..step 1
pickSoftThreshold: will use block size 1680.
 pickSoftThreshold: calculating connectivity for given powers... 
   Power SFT.R.sq  slope truncated.R.sq  mean.k. median.k. max.k.
1      1    0.041 -0.346          0.905 4250.000  4.08e+03 7890.0
2      2    0.622 -1.190          0.857 1210.000  9.79e+02 3620.0
3      3    0.700 -1.330          0.785  484.000  2.95e+02 2020.0
4      4    0.845 -1.160          0.849  240.000  1.03e+02 1270.0
5      5    0.862 -1.250          0.903  138.000  4.01e+01  992.0
6      6    0.876 -1.280          0.942   86.900  1.69e+01  811.0
7      7    0.889 -1.300          0.964   58.800  7.56e+00  677.0
8      8    0.891 -1.310          0.972   41.800  3.55e+00  574.0
9      9   

[1] "male"

...

[1] "male"

Power,SFT.R.sq,slope,truncated.R.sq,mean.k.,median.k.,max.k.
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.03047936,-0.3016302,0.9014501,3906.0684497,3.760008e+03,7581.18358
2,0.58590224,-1.2699595,0.8324474,1050.0188930,8.349647e+02,3354.22326
3,0.65271084,-1.4220954,0.7595869,402.1162883,2.346538e+02,1820.50292
4,0.86184254,-1.1989487,0.8868867,193.5789941,7.665791e+01,1132.11518
5,0.86368145,-1.3082959,0.9281571,108.3874791,2.789131e+01,889.51475
6,0.88295808,-1.3340869,0.9577294,67.2188709,1.111324e+01,720.85044
7,0.88006477,-1.3520310,0.9651765,44.7767403,4.715439e+00,596.69217
8,0.88778697,-1.3513821,0.9727314,31.4150278,2.131939e+00,501.88529
9,0.89189733,-1.3576999,0.9778847,22.9161592,9.950253e-01,427.55846


[1] "female"

Power,SFT.R.sq,slope,truncated.R.sq,mean.k.,median.k.,max.k.
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.04675225,-0.4476371,0.9205575,4370.4399339,4.315119e+03,7957.79238
2,0.53249310,-1.2535674,0.8799631,1240.0759427,1.081469e+03,3522.83477
3,0.64699252,-1.4010752,0.8194740,480.6678149,3.380529e+02,1892.41218
4,0.83536461,-1.2675615,0.9063818,228.9759270,1.222293e+02,1170.72093
5,0.85928134,-1.4139920,0.9482732,125.7492711,4.860109e+01,911.52696
6,0.88322972,-1.4484175,0.9700858,76.3657669,2.086091e+01,731.66201
7,0.89390735,-1.4526974,0.9794449,49.8819437,9.562496e+00,600.19310
8,0.88895651,-1.4582173,0.9749824,34.3977044,4.577396e+00,500.62414
9,0.88884809,-1.4538698,0.9727246,24.7221949,2.281323e+00,423.18229


   Power SFT.R.sq  slope truncated.R.sq  mean.k. median.k. max.k.
1      1   0.0468 -0.448          0.921 4370.000  4.32e+03 7960.0
2      2   0.5320 -1.250          0.880 1240.000  1.08e+03 3520.0
3      3   0.6470 -1.400          0.819  481.000  3.38e+02 1890.0
4      4   0.8350 -1.270          0.906  229.000  1.22e+02 1170.0
5      5   0.8590 -1.410          0.948  126.000  4.86e+01  912.0
6      6   0.8830 -1.450          0.970   76.400  2.09e+01  732.0
7      7   0.8940 -1.450          0.979   49.900  9.56e+00  600.0
8      8   0.8890 -1.460          0.975   34.400  4.58e+00  501.0
9      9   0.8890 -1.450          0.973   24.700  2.28e+00  423.0
10    10   0.8940 -1.440          0.975   18.400  1.18e+00  362.0
11    11   0.8920 -1.440          0.974   14.000  6.31e-01  312.0
12    12   0.8920 -1.440          0.973   10.900  3.46e-01  271.0
13    13   0.8830 -1.450          0.965    8.620  1.94e-01  237.0
14    14   0.8820 -1.440          0.965    6.930  1.12e-01  209.0
15    15  

In [11]:
scale_free_df2

Power,SFT.R.sq,slope,truncated.R.sq,mean.k.,median.k.,max.k.,brain_region,sex,status
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
15,0.8862898,-1.398608,0.9767316,5.513155,0.02037328,194.7508,dlpfc,male,CTL_SZD
15,0.8889554,-1.437447,0.9697805,5.639542,0.06592270,185.1673,dlpfc,female,CTL_SZD
15,0.8676044,-1.488453,0.9595083,4.426547,0.04846326,162.3829,dlpfc,male,SZD
15,0.8794784,-1.428097,0.9370866,6.418598,0.19990230,235.2397,dlpfc,female,SZD
15,0.9020769,-1.344196,0.9887057,7.865503,0.04420381,233.7331,dlpfc,male,CTL
15,0.8616363,-1.504724,0.9823709,8.429590,0.22049679,198.1165,dlpfc,female,CTL


# Main Results

In [12]:
# with these 3 DLPFC samples removed (R3555, R12351, R12371)  we were able to get a scale-free network also on Power 15 (R2 >= 0.85) on all DLPFC data as well

metadata %>% 
        filter(RNum == 'R3555' | RNum == 'R12351' | RNum == 'R12371') %>%
        select(RNum, Sex, Dx, RIN, Region)

,RNum,Sex,Dx,RIN,Region
,<chr>,<fct>,<fct>,<dbl>,<chr>
R12351,R12351,F,Control,6.5,DLPFC
R12371,R12371,F,SCZD,6.2,DLPFC
R3555,R3555,M,SCZD,9.0,DLPFC


In [13]:
options(repr.matrix.max.cols=100, repr.matrix.max.rows=100)

metadata %>% 
        filter(RNum == 'R3555' | RNum == 'R12351' | RNum == 'R12371') 

,group,lib.size,norm.factors,SAMPLE_ID,RNum,Region,Dataset,Protocol,RIN,BrNum,Dx,Race,Sex,Age,PMI,MoD,Mapping_Rate,Base_Mismatch,ExprProfEff,Exonic_Rate,Intronic_Rate,Intergenic_Rate,totalAssignedGene,Ambiguous_Alignment_Rate,rRNA_rate,End_1_Sense_Rate,End_2_Sense_Rate,Chimeric_Alignment_Rate,Low_Mapping_Quality,Low_Quality_Reads,Genes_Detected,Mean3Bias,numReads,totalMapped,Mapped_Unique_Reads,Intergenic_Reads,gene_Assigned,Ambiguous_Reads,Intronic_Reads,Read_Length,NonGlobin_Reads,MedianAvgTxCov,Mito_mapped,mitoRate,globinRate,IID,SOL,snpPC1,snpPC2,snpPC3,snpPC4,snpPC5,snpPC6,snpPC7,snpPC8,snpPC9,snpPC10,New_Dx,antipsychotics,lifetime_antipsych
,<fct>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<fct>,<chr>,<fct>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<lgl>
R12351,1,54780010,0.9441608,R12351_C5AEJACXX,R12351,DLPFC,BrainSeq_Phase2_DLPFC,RiboZeroGold,6.5,Br1289,Control,AA,F,17.27,15.5,Homicide,0.980549,0.00225312,0.348979,0.355901,0.558159,0.0461803,0.914061,0.0397591,0.00853538,0.0640477,0.933025,0.00545214,19508567,22497212,29553,0.530782,247598326,158487807,158487807,7319007,144867462,6301338,88461448,100,158485859,3.39481,2445098,0.009875261,7.867581e-06,4040296051_A,0,0.0775027,-0.00377709,-1.64760e-04,0.00130565,0.001503210,-0.000443292,0.000676359,0.000179677,-0.000669624,-0.000142894,Control,FALSE,FALSE
R12371,1,46147434,1.0431421,R12371_C5ADGACXX,R12371,DLPFC,BrainSeq_Phase2_DLPFC,RiboZeroGold,6.2,Br1420,SCZD,AA,F,80.20,17.5,Natural,0.979922,0.00207988,0.325075,0.331735,0.563556,0.0637595,0.895292,0.0409487,0.00607702,0.0482978,0.950043,0.00584187,17133187,18712553,29836,0.540818,215833337,126977892,126977892,8096053,113682258,5199581,71559197,100,126973167,3.37159,1501382,0.006956210,2.189189e-05,4572348703_R01C01,0,0.0673397,-0.00459136,1.30164e-03,-0.00139981,0.000280948,-0.000426639,-0.001390620,-0.002325950,-0.002642880,-0.000611504,Schizo_noAP,FALSE,TRUE
R3555,1,46409298,1.7344757,R3555_C4KHUACXX,R3555,DLPFC,BrainSeq_Phase2_DLPFC,RiboZeroGold,9.0,Br1427,SCZD,CAUC,M,66.81,24.5,Natural,0.951226,0.00293084,0.421507,0.443120,0.472219,0.0502190,0.915339,0.0344419,0.00901134,0.0526278,0.945205,0.00800872,12140717,13390776,29384,0.599414,139795817,99833469,99833469,5013540,91381474,3438455,47143277,100,99831003,2.98411,1825485,0.013058223,1.764001e-05,4584656109_R01C02,0,-0.0222989,-0.00267137,7.06334e-05,0.00703864,-0.011538200,0.007701430,-0.002297560,0.002068600,0.001219660,-0.001039860,Schizo_AP,TRUE,TRUE


### Scale free topology index final results from Power 15

In [14]:
#Caudate & Hippocampus
scale_free_df %>% filter(Power == 15) %>% filter(brain_region != 'dlpfc') 

Power,SFT.R.sq,slope,truncated.R.sq,mean.k.,median.k.,max.k.,brain_region,sex,status
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
15,0.8603353,-1.649639,0.9721536,1.2683011,0.0356344068,43.80547,hippocampus,male,CTL_SZD
15,0.9129470,-1.555580,0.9899227,2.8848024,0.0712492655,114.24422,hippocampus,female,CTL_SZD
15,0.8998215,-1.720211,0.9895271,1.6997666,0.0819433563,72.02038,hippocampus,male,SZD
15,0.8667655,-1.800413,0.9775726,3.3379508,0.2572405980,133.03247,hippocampus,female,SZD
15,0.8503683,-1.750640,0.9813398,1.5508506,0.0557427110,51.48229,hippocampus,male,CTL
15,0.8814454,-1.569738,0.9728334,4.7189026,0.1549152156,151.23476,hippocampus,female,CTL
15,0.9073596,-1.620669,0.9797628,0.1047947,0.0003920376,12.71824,caudate,male,CTL_SZD
15,0.9603098,-1.573757,0.9859210,0.1235040,0.0022253068,17.47407,caudate,female,CTL_SZD
15,0.9423368,-1.624786,0.9928773,0.1490048,0.0029376827,12.28523,caudate,male,SZD


In [15]:
#DLPFC
scale_free_df2

Power,SFT.R.sq,slope,truncated.R.sq,mean.k.,median.k.,max.k.,brain_region,sex,status
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
15,0.8862898,-1.398608,0.9767316,5.513155,0.02037328,194.7508,dlpfc,male,CTL_SZD
15,0.8889554,-1.437447,0.9697805,5.639542,0.06592270,185.1673,dlpfc,female,CTL_SZD
15,0.8676044,-1.488453,0.9595083,4.426547,0.04846326,162.3829,dlpfc,male,SZD
15,0.8794784,-1.428097,0.9370866,6.418598,0.19990230,235.2397,dlpfc,female,SZD
15,0.9020769,-1.344196,0.9887057,7.865503,0.04420381,233.7331,dlpfc,male,CTL
15,0.8616363,-1.504724,0.9823709,8.429590,0.22049679,198.1165,dlpfc,female,CTL


In [16]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Arch Linux

Matrix products: default
BLAS:   /usr/lib/libopenblas.so.0.3
LAPACK: /usr/lib/liblapack.so.3.11.0

locale:
 [1] LC_CTYPE=C.UTF-8    LC_NUMERIC=C        LC_TIME=C          
 [4] LC_COLLATE=C        LC_MONETARY=C       LC_MESSAGES=C      
 [7] LC_PAPER=C          LC_NAME=C           LC_ADDRESS=C       
[10] LC_TELEPHONE=C      LC_MEASUREMENT=C    LC_IDENTIFICATION=C

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] limma_3.54.2          IRdisplay_1.1         gridExtra_2.3        
 [4] latticeExtra_0.6-30   lattice_0.20-45       flashClust_1.01-2    
 [7] WGCNA_1.72-1          fastcluster_1.2.3     dynamicTreeCut_1.63-1
[10] data.table_1.14.8     lubridate_1.9.2       forcats_1.0.0        
[13] stringr_1.5.0         dplyr_1.1.0           purrr_1.0.1          
[16] readr_2.1.4           tidyr_1.3.0      